In [1]:
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/My Drive/data/food.zip' '/content/food.zip'
!unzip *.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: food/0_564.png          
  inflating: food/0_565.png          
  inflating: food/0_566.png          
  inflating: food/0_567.png          
  inflating: food/0_568.png          
  inflating: food/0_569.png          
  inflating: food/0_57.png           
  inflating: food/0_570.png          
  inflating: food/0_571.png          
  inflating: food/0_572.png          
  inflating: food/0_573.png          
  inflating: food/0_574.png          
  inflating: food/0_575.png          
  inflating: food/0_576.png          
  inflating: food/0_577.png          
  inflating: food/0_578.png          
  inflating: food/0_579.png          
  inflating: food/0_58.png           
  inflating: food/0_580.png          
  inflating: food/0_581.png          
  inflating: food/0_582.png          
  inflating: food/0_583.png          
  inflating: food/0_584.png          
  inflating: food/0_585.png          
  inflating: food/0_

In [ ]:
model_resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
for name, param in model_resnet18.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

num_classes = 2
model_resnet18.fc = nn.Sequential(nn.Linear(model_resnet18.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

Downloading: "https://github.com/pytorch/vision/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
def train(model, optimizer, sheduler, loss_fn, train_loader, val_loader, epochs=5, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        n_train = len(train_loader)
        for batch in tqdm(train_loader, total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img'):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        scheduler.step()

        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in tqdm(val_loader, total=len(val_loader), desc=f' Validation Epoch {epoch + 1}/{epochs}', unit='img'):
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
                        
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.4f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [5]:
img_dimensions = 224

img_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

In [ ]:
class dataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        label = img_path.split('/')[-1].split('_')[0]
        if label == '0':
            label = 0
        elif label == '1':
            label = 1
            
        return img_transformed,label

In [ ]:
import os
import glob
import random

train_dir = "/content/food/"
train_list = glob.glob(os.path.join(train_dir,'*.png'))
random.shuffle(train_list)
train_list, val_list = train_test_split(train_list, test_size=0.2)

In [ ]:
train_list[0:100]

['/content/food/0_1668.png',
 '/content/food/0_3570.png',
 '/content/food/1_2046.png',
 '/content/food/0_1525.png',
 '/content/food/1_2917.png',
 '/content/food/1_5366.png',
 '/content/food/0_2972.png',
 '/content/food/1_7324.png',
 '/content/food/0_4379.png',
 '/content/food/1_730.png',
 '/content/food/0_3738.png',
 '/content/food/1_2351.png',
 '/content/food/0_4076.png',
 '/content/food/0_5078.png',
 '/content/food/0_2164.png',
 '/content/food/1_3035.png',
 '/content/food/1_789.png',
 '/content/food/0_2307.png',
 '/content/food/1_3593.png',
 '/content/food/0_3063.png',
 '/content/food/0_13.png',
 '/content/food/1_4129.png',
 '/content/food/0_4241.png',
 '/content/food/1_1808.png',
 '/content/food/1_1555.png',
 '/content/food/1_4428.png',
 '/content/food/0_319.png',
 '/content/food/1_6146.png',
 '/content/food/0_3235.png',
 '/content/food/0_5199.png',
 '/content/food/0_1368.png',
 '/content/food/0_555.png',
 '/content/food/1_4246.png',
 '/content/food/1_4788.png',
 '/content/food/0_72

In [ ]:
train_data = dataset(train_list, transform=img_transforms)
val_data = dataset(val_list, transform=img_transforms)
                   
num_workers = 6
batch_size = 32

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [ ]:
model_resnet18.to(device)
optimizer = optim.AdamW(model_resnet18.parameters(), lr=3e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
train(model_resnet18, optimizer, scheduler, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=2, device=device)

Epoch 2/2:   0%|          | 0/245 [00:00<?, ?img/s]

Epoch: 0, Training Loss: 0.2197, Validation Loss: 0.1228, accuracy = 0.9587


 Validation Epoch 2/2: 100%|██████████| 62/62 [00:15<00:00,  3.93img/s]

Epoch: 1, Training Loss: 0.1505, Validation Loss: 0.1435, accuracy = 0.9546


In [ ]:
torch.save(model_resnet18.state_dict(), "/content/drive/My Drive/data/resnet18.pth")

In [4]:
resnet_loaded = torch.hub.load('pytorch/vision', 'resnet18')
resnet_loaded.fc = nn.Sequential(nn.Linear(resnet_loaded.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, 2))
resnet_loaded.load_state_dict(torch.load('/content/drive/My Drive/data/resnet18.pth'))
resnet_loaded.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_master


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
    filename = '/content/food/0_686.png'
    labels = ['nonfood', 'food']
    img = Image.open(filename)
    img = img_transforms(img)
    img = img.unsqueeze(0)
    resnet_loaded.to(device)
    prediction = resnet_loaded(img.to(device))
    prediction = prediction.argmax()
    print(labels[prediction])

food
